# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

## Выполним задание с использованием модуля Detoxify (toxic Bert). 
Модуль на входе получает ощищенный текст, назад возвращает некоторые оценки токсичности, основанные на 7 параметрах: 'toxicity', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat', 'sexual_explicit'.

In [1]:
!pip install pip install detoxify

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.0 MB/s 
     |████████████████████████████████| 4.7 MB 46.5 MB/s 
     |████████████████████████████████| 6.6 MB 53.4 MB/s 
     |████████████████████████████████| 120 kB 65.9 MB/s 


In [4]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

import re
import torch
from detoxify import Detoxify
from tqdm.notebook import tqdm

In [5]:
torch.cuda.is_available()

True

In [6]:
from google.colab import drive
drive.mount('/content/drive')

data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/toxic_comments.csv', on_bad_lines='skip')

Mounted at /content/drive


## Подготовка данных

In [7]:
#data.info()
#data.toxic.unique()
data.head(3)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0


### С данными все в порядке: Целевой признак в числовом формате, уникальные значения 0 и 1. Необходимо только удалить столбец 'Unnamed: 0'.

In [8]:
data = data.drop(columns='Unnamed: 0')

## Проведем предобработку данных 
Удалим ненужные символы, приведем текст к нижнему регистру.

In [22]:
def clear_text(text):
    return " ".join(re.sub(r"[^a-zA-Z']", " ", text).split()) #есть смысл добавить апостроф ('), т.к. don't, can't

data['text'] = data.text.apply(clear_text)

In [10]:
data['text'].head()

0    Explanation Why the edits made under my userna...
1    D'aww He matches this background colour I'm se...
2    Hey man I'm really not trying to edit war It's...
3    More I can't make any real suggestions on impr...
4    You sir are my hero Any chance you remember wh...
Name: text, dtype: object

### Проверим работу модели на тексте, напишем функцию, которую можно будет применить ко всему датафрейму.

In [11]:
model = Detoxify('original', device='cuda')

Downloading: "https://github.com/unitaryai/detoxify/releases/download/v0.1-alpha/toxic_original-c1212f89.ckpt" to /root/.cache/torch/hub/checkpoints/toxic_original-c1212f89.ckpt


  0%|          | 0.00/418M [00:00<?, ?B/s]

In [19]:
model.predict('what a great day is today')

{'toxicity': 0.0016122606,
 'severe_toxicity': 9.4457304e-05,
 'obscene': 0.0001897773,
 'threat': 0.00010733206,
 'insult': 0.0001974744,
 'identity_attack': 0.00015127673}

In [20]:
def new_features(x):
    prediction = model.predict(x['text'])
    return prediction['toxicity'], prediction['severe_toxicity'], prediction['obscene'], prediction['identity_attack'], prediction['insult'], prediction['threat']

In [21]:
%%time
tqdm.pandas()
df_features = data.progress_apply(new_features, axis=1, result_type='expand')
df_features.columns = ['toxicity', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']

  0%|          | 0/159292 [00:00<?, ?it/s]

CPU times: user 36min 1s, sys: 17.8 s, total: 36min 19s
Wall time: 36min 41s


### После преобразования столбца с текстом, у нас получился следующий датасет с признаками:

In [23]:
df_features.head(3)

,toxicity,severe_toxicity,obscene,identity_attack,insult,threat
0,0.000961,0.000100,0.000187,0.000135,0.000181,0.000109
1,0.000778,0.000109,0.000176,0.000135,0.000181,0.000121
2,0.001145,0.000098,0.000189,0.000140,0.000182,0.000109


#### Разобьем выборку на тренировочную, валидационную и тестовую, подберем оптимальные параметры логической регрессии, максимизируем F1.

In [25]:
df_features_train, df_features_test, df_target_train, df_target_test = train_test_split(df_features, data['toxic'], test_size=0.2, random_state=12345)
df_features_train, df_features_val, df_target_train, df_target_val = train_test_split(df_features_train, df_target_train, test_size=0.25, random_state=12345)

In [26]:
%%time
model = LogisticRegression(max_iter=1000)
hyperparameters = {
                    'class_weight': [None, 'balanced'],
                    'C': [0.1, 1, 10]
                  }

clf = GridSearchCV(model, param_grid = hyperparameters, cv=3, scoring='roc_auc', n_jobs=-1)
 
# Fit and tune model
clf.fit(df_features_train, df_target_train)
clf.best_params_

CPU times: user 557 ms, sys: 345 ms, total: 901 ms
Wall time: 4.6 s


{'C': 0.1, 'class_weight': 'balanced'}

In [27]:
#refitting on entire training data using best settings
clf.refit
probabilities_valid = clf.predict_proba(df_features_val)
probabilities_one_valid = probabilities_valid[:, 1]

In [28]:
f1_max = 0
threshold_opt = None

for threshold in np.arange(0, 1, 0.02):
    predicted_valid = probabilities_one_valid > threshold 
    f1 = f1_score(predicted_valid, df_target_val)
    if f1>f1_max:
        f1_max=f1
        threshold_opt = threshold
        
print (f1_max, threshold_opt) 

0.9173771508293287 0.9


In [29]:
probabilities_test = clf.predict_proba(df_features_test)
probabilities_one_test = probabilities_test[:, 1]
predicted_test = probabilities_one_test > 0.2
f1_score(predicted_test, df_target_test)

0.8841105836358559